In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 

from peripheralManager import PeripheralManager
from components.CellVisualizer.visualizer import CellVisualizer
from components.cellExtractor.extractor import cellExtractor

peripheral_manager = PeripheralManager()
cell_visualizer = CellVisualizer()

c:\Users\JCardeteLl\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
directory1 = "C:/Users/JCardeteLl/Documents/TFG/Bases_de_datos/images_x100_cellavision_fundacion"
directory2 = "C:/Users/JCardeteLl/Documents/TFG/Bases_de_datos/images_x60_reales_fundacion"


# Create a dataset
dataset_100 = tf.keras.preprocessing.image_dataset_from_directory(
    directory1,
    image_size=(360, 363),  # Resize images to this size
    batch_size=32,          # Number of images to return in each batch
    label_mode='int'        # Labels are returned as integers (you can also use 'categorical' or 'binary')
)

dataset_60 = tf.keras.preprocessing.image_dataset_from_directory(
    directory2,
    image_size=(360, 363),  # Resize images to this size
    batch_size=32,          # Number of images to return in each batch
    label_mode='int'        # Labels are returned as integers (you can also use 'categorical' or 'binary')
)

Found 1328 files belonging to 7 classes.
Found 99 files belonging to 10 classes.


In [3]:
def category_count(dataset):
        category_counts = {}

        # Iterate over the dataset
        for images, labels in dataset:
            for label in labels.numpy():  # Convert labels to numpy array for processing
                if label in category_counts:
                    category_counts[label] += 1
                else:
                    category_counts[label] = 1

        # Create a dictionary to store the counts with class names as keys
        class_name_counts = {}
        class_names = dataset.class_names
        for label, count in category_counts.items():
            class_name_counts[class_names[label]] = count

        return class_name_counts


def get_sample_images(name: str, num_images: int, dataset) -> list:
    """
    Collects a specified number of images from the dataset that match the given class name.
    
    Args:
    name (str): The name of the class to find images of.
    num_images (int): The number of images to collect.

    Returns:
    List[np.ndarray]: A list of images as numpy arrays.
    """
    target_index = dataset.class_names.index(name)
    collected_images = []

    for images, labels in dataset.take(num_images):  # Adjust the number of batches taken if needed
        for image, label in zip(images, labels):
            if label == target_index and len(collected_images) < num_images:
                collected_images.append(image.numpy().astype("uint8"))
            if len(collected_images) == num_images:
                break
        if len(collected_images) == num_images:
            break

    return collected_images

def sample_visualize(sample_images: list, rows: int, columns: int, cmap: str = None):
    """
    Visualizes a sample of images from the dataset that match the given class name in a grid format.

    Args:
    sample_images (list): List of images to visualize.
    rows (int): The number of rows in the image grid.
    columns (int): The number of columns in the image grid.
    cmap (str, optional): Color map to use for displaying images.
    """
    max_images = rows * columns  # Define the maximum number of images to display
    fig, axes = plt.subplots(rows, columns, figsize=(5 * columns, 5 * rows))
    axes = axes.flatten()  # Flatten the array of axes to make iterating easier

    for ax, img in zip(axes, sample_images[:max_images]):
        ax.imshow(img, cmap=cmap)
        ax.axis('off')

    # Turn off any unused axes
    for ax in axes[len(sample_images):]:
        ax.axis('off')

    plt.show()

In [ ]:
def main_process_visualizer(sample: np.ndarray):
    grayscale_sample = [peripheral_manager.convert_grayscale(image = line, converter="simple") for line in sample]  

    binarymask_sample = [peripheral_manager.binary_mask(grayscale_image = line, masker="adaptative") for line in grayscale_sample] 

    locations_sample =  [peripheral_manager.cell_localizers(binary_image = line, localizer="simple") for line in binarymask_sample] 

    cellMasker_sample = [peripheral_manager.size_mask(locations = line, masker = "SizeMasker") for line in locations_sample]

    filtered_locations_sample = [cell_visualizer.visualize_bounding_boxes(image = line1, bounding_boxes = line2, classifications = ["tricholeukemia"]*8) for line1, line2 in zip(sample, cellMasker_sample)]

    return grayscale_sample, binarymask_sample, locations_sample, cellMasker_sample, filtered_locations_sample



def main_process_visualizer_single(image: np.ndarray):
    grayscale_sample = peripheral_manager.convert_grayscale(image = image, 
                                                            converter="simple")

    binarymask_sample = peripheral_manager.binary_mask(grayscale_image = grayscale_sample, 
                                                       masker="adaptative")

    locations_sample =  peripheral_manager.cell_localizers(binary_image = binarymask_sample, 
                                                           localizer="simple")

    cellMasker_sample = peripheral_manager.size_mask(locations = locations_sample, 
                                                     masker = "SizeMasker")

    filtered_locations_sample = cell_visualizer.visualize_bounding_boxes(image = image, 
                                                                         bounding_boxes = cellMasker_sample, 
                                                                         classifications = ["tricholeukemia"]*len(cellMasker_sample))

    return grayscale_sample, binarymask_sample, locations_sample, cellMasker_sample, filtered_locations_sample

In [ ]:
count_dict = category_count(dataset_60)
cells_dict = {}
cells_process={}

for class_name, count in count_dict.items():
    cells_dict[class_name] = get_sample_images(class_name, count, dataset_60)
    cells_process[class_name] = main_process_visualizer(cells_dict[class_name])

In [ ]:
print(count_dict)

In [ ]:
sample_visualize(cells_process["Linfocitos normales"][4], 1, 4)

In [ ]:
sample_visualize(cells_process["Leucemia aguda linfoblastica"][4], 1, 3)

In [ ]:
sample_visualize(cells_process["Linfocitos vellosos"][4], 1, 7)

In [ ]:
sample_visualize(cells_process["Linfoma del manto"][4], 1, 6)

In [ ]:
sample_visualize(cells_process["LLC"][4], 1, 2)

In [ ]:
sample_visualize(cells_process["Plasmatica"][4], 1, 5)

In [ ]:
sample_visualize(cells_process["Sezary"][4], 1, 5)

In [ ]:
sample_visualize(cells_process["Tricholeukemia"][4], 3, 4)

In [ ]:
image = plt.imread(r"C:\Users\JCardeteLl\Documents\TFG\Bases_de_datos\images_x60_reales_fundacion\Marginal\IMG_7863.jpg")

image_processed = main_process_visualizer_single(image)


fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (20, 8))

axes[0].imshow(image_processed[0], cmap = "gray")
axes[0].axis("off")

axes[1].imshow(image_processed[1], cmap = "gray")
axes[1].axis("off")

axes[2].imshow(image_processed[4])
axes[2].axis("off")

plt.show()


In [ ]:
list_images = get_sample_images("Leucemia aguda linfoblastica", 4, dataset_60)

In [ ]:
plt.imshow(list_images[0])
plt.axis("off")
plt.show()

In [ ]:
grayscale_sample = peripheral_manager.convert_grayscale(image = image, 
                                                            converter="simple")

In [ ]:
plt.imshow(grayscale_sample, cmap = "gray")
plt.axis("off")
plt.show()

In [ ]:
binarymask_sample = peripheral_manager.binary_mask(grayscale_image = grayscale_sample, 
                                                       masker="adaptative")

In [ ]:
plt.imshow(binarymask_sample, cmap = "gray")
plt.axis("off")
plt.show()

In [ ]:
image = plt.imread(r"C:\Users\JCardeteLl\Documents\TFG\Bases_de_datos\images_x60_reales_fundacion\Leucemia aguda linfoblastica\image3.jpg")

In [ ]:
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
grayscale_sample = peripheral_manager.convert_grayscale(image = image, 
                                                            converter="simple")

In [ ]:
plt.imshow(grayscale_sample, cmap = "gray")
plt.axis("off")
plt.show()

In [ ]:
binarymask_sample = peripheral_manager.binary_mask(grayscale_image = grayscale_sample, 
                                                       masker="adaptative")

In [ ]:
plt.imshow(binarymask_sample, cmap = "gray")
plt.axis("off")
plt.show()

In [ ]:
def main_process_visualizer_single(image: np.ndarray):
    grayscale_sample = peripheral_manager.convert_grayscale(image = image, 
                                                            converter="simple")

    binarymask_sample = peripheral_manager.binary_mask(grayscale_image = grayscale_sample, 
                                                       masker="adaptative")

    locations_sample =  peripheral_manager.cell_localizers(binary_image = binarymask_sample, 
                                                           localizer="simple")

    cellMasker_sample = peripheral_manager.size_mask(locations = locations_sample, 
                                                     masker = "SizeMasker")

    filtered_locations_sample = cell_visualizer.visualize_bounding_boxes(image = image, 
                                                                         bounding_boxes = cellMasker_sample, 
                                                                         classifications = ["tricholeukemia"]*len(cellMasker_sample))

    return grayscale_sample, binarymask_sample, locations_sample, cellMasker_sample, filtered_locations_sample